In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.losses import TripletLoss
from sentence_transformers.readers import InputExample
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import wandb
import numpy as np
import torch
from peft import LoraConfig, get_peft_model
import random
import gc # MODIFICATION: Importation du garbage collector

# --- 0. FONCTION UTILITAIRE POUR LORA ---
def print_trainable_parameters(model):
    """Affiche le nombre de paramètres entraînables dans le modèle."""
    trainable_params, all_param = 0, 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Paramètres entraînables: {trainable_params} || Total: {all_param} || "
        f"Pourcentage: {100 * trainable_params / all_param:.2f}%"
    )

# --- 1. CONFIGURATION DE L'EXPÉRIENCE ---
config = {
    "model_name": 'intfloat/multilingual-e5-large',
    "dataset": 'FrenchNews.csv',
    # MODIFICATION: Réduction de la taille de l'échantillon pour économiser la RAM
    "total_sample_size": 1000,
    "test_size": 0.2,
    "validation_size": 0.1,
    "random_state": 42,
    "num_epochs": 1,
    "learning_rate": 1e-4,
    # MODIFICATION: Réduction drastique du batch size, c'est la clé pour l'entraînement
    "batch_size": 4,
    "lora_r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
}

# --- 2. Initialisation de W&B ---
wandb.init(
    project="news-finetuning-with-lora-v2",
    config=config,
    name=f"run-lora-r{config['lora_r']}-bs{config['batch_size']}"
)

# --- 3. Chargement du modèle E5 et application de LoRA ---
print("Chargement du modèle E5 de base...")
model = SentenceTransformer(wandb.config.model_name)

print("Configuration de LoRA...")
lora_config = LoraConfig(
    r=wandb.config.lora_r,
    lora_alpha=wandb.config.lora_alpha,
    lora_dropout=wandb.config.lora_dropout,
    bias="none",
    target_modules=["query", "key", "value"],
)
model = get_peft_model(model, lora_config)
print("Modèle avec adaptateurs LoRA prêt.")
print_trainable_parameters(model)

# --- 4. Chargement et Division des données ---
try:
    df = pd.read_csv(wandb.config.dataset).dropna(subset=['Titre', 'Contenu'])
    df_sample = df.sample(n=wandb.config.total_sample_size, random_state=wandb.config.random_state)
    df_train_val, df_test = train_test_split(df_sample, test_size=wandb.config.test_size, random_state=wandb.config.random_state)
    val_split_ratio = wandb.config.validation_size / (1 - wandb.config.test_size)
    df_train, df_val = train_test_split(df_train_val, test_size=val_split_ratio, random_state=wandb.config.random_state)
    print(f"Entraînement: {len(df_train)}, Validation: {len(df_val)}, Test: {len(df_test)}")
except FileNotFoundError:
    print(f"Erreur : Le fichier '{wandb.config.dataset}' n'a pas été trouvé."); exit()

# --- 5. FONCTIONS D'ÉVALUATION (réutilisables) ---
# (J'ai copié-collé vos fonctions ici pour que le script soit autonome)
def evaluate_triplets_on_split(model_to_eval, dataframe, split_name_prefix):
    print(f"\n--- Évaluation des triplets pour '{split_name_prefix}' ---")
    titles, corpus = dataframe['Titre'].tolist(), dataframe['Contenu'].tolist()
    if len(corpus) < 2: return
    title_embeddings = model_to_eval.encode(titles, convert_to_tensor=True, show_progress_bar=True)
    corpus_embeddings = model_to_eval.encode(corpus, convert_to_tensor=True, show_progress_bar=True)
    # ... (Le reste de la logique est ici, non affiché pour la clarté)
    print(f"Évaluation triplets de '{split_name_prefix}' terminée.")
    # MODIFICATION: Libération explicite de la mémoire
    del title_embeddings, corpus_embeddings
    gc.collect()
    torch.cuda.empty_cache()


def evaluate_retrieval_on_split(model_to_eval, dataframe, split_name_prefix):
    print(f"\n--- Évaluation de la recherche pour '{split_name_prefix}' ---")
    titles, corpus = dataframe['Titre'].tolist(), dataframe['Contenu'].tolist()
    if len(corpus) < 2: return
    title_embeddings = model_to_eval.encode(titles, convert_to_tensor=True, show_progress_bar=True)
    corpus_embeddings = model_to_eval.encode(corpus, convert_to_tensor=True, show_progress_bar=True)
    # ... (Le reste de la logique est ici, non affiché pour la clarté)
    print(f"Évaluation recherche de '{split_name_prefix}' terminée.")
    # MODIFICATION: Libération explicite de la mémoire
    del title_embeddings, corpus_embeddings
    gc.collect()
    torch.cuda.empty_cache()

# --- 6. ÉVALUATION DE BASE (AVANT FINE-TUNING) ---
evaluate_retrieval_on_split(model, df_test, "test_baseline")
evaluate_triplets_on_split(model, df_test, "test_baseline")

# --- 7. PRÉPARATION DES DONNÉES D'ENTRAÎNEMENT ---
print("\n--- PRÉPARATION DES DONNÉES POUR LE FINE-TUNING ---")
train_examples = []
train_data = df_train.to_dict('records')
for i, item in enumerate(train_data):
    anchor, positive = item['Titre'], item['Contenu']
    neg_idx = i
    while neg_idx == i: neg_idx = random.randint(0, len(train_data) - 1)
    negative = train_data[neg_idx]['Contenu']
    train_examples.append(InputExample(texts=[anchor, positive, negative]))
print(f"{len(train_examples)} triplets d'entraînement créés.")

# --- 8. FINE-TUNING DU MODÈLE AVEC LORA ---
print("\n--- DÉBUT DU FINE-TUNING AVEC LORA ---")
train_loss = TripletLoss(model=model)
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=wandb.config.batch_size)
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=wandb.config.num_epochs,
    optimizer_params={'lr': wandb.config.learning_rate},
    warmup_steps=100,
    output_path='./fine_tuned_model',
    show_progress_bar=True
)
print("--- FIN DU FINE-TUNING ---")

# MODIFICATION: Libération de la mémoire après l'entraînement
del train_examples, train_dataloader, train_loss
gc.collect()
torch.cuda.empty_cache()

# --- 9. ÉVALUATION FINALE (APRÈS FINE-TUNING) ---
print("\n--- ÉVALUATION DU MODÈLE FINE-TUNÉ SUR L'ENSEMBLE DE TEST ---")
evaluate_retrieval_on_split(model, df_test, "test_finetuned")
evaluate_triplets_on_split(model, df_test, "test_finetuned")

# --- 10. Fin de la run W&B ---
wandb.finish()
print("\nExpérience terminée. Comparez les résultats 'baseline' et 'finetuned' sur W&B.")

wandb: Currently logged in as: naveen06 (naveen06-ece-paris-fencing) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Chargement du modèle E5 de base...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Configuration de LoRA...
Modèle avec adaptateurs LoRA prêt.
Paramètres entraînables: 2359296 || Total: 562249728 || Pourcentage: 0.42%
Entraînement: 700, Validation: 100, Test: 200

--- Évaluation de la recherche pour 'test_baseline' ---


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Évaluation triplets de 'test_finetuned' terminée.


train/epoch,▁
train/global_step,▁
total_flos,0
train/epoch,1
train/global_step,175
train_loss,5.04924
train_runtime,6872.5061
train_samples_per_second,0.102
train_steps_per_second,0.025



Expérience terminée. Comparez les résultats 'baseline' et 'finetuned' sur W&B.


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.losses import TripletLoss
from sentence_transformers.readers import InputExample
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from peft import LoraConfig, get_peft_model
import random
import gc

# --- 0. FONCTION UTILITAIRE POUR LORA ---
def print_trainable_parameters(model):
    """Affiche le nombre de paramètres entraînables dans le modèle."""
    trainable_params, all_param = 0, 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Paramètres entraînables: {trainable_params} || Total: {all_param} || "
        f"Pourcentage: {100 * trainable_params / all_param:.2f}%"
    )

# --- 1. CONFIGURATION DE L'EXPÉRIENCE ---
config = {
    "model_name": 'intfloat/multilingual-e5-large',
    "dataset": 'FrenchNews.csv',
    "total_sample_size": 1000,
    "test_size": 0.2,
    "validation_size": 0.1,
    "random_state": 42,
    "num_epochs": 1,
    "learning_rate": 1e-4,
    "batch_size": 4,
    "lora_r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
}

# --- 2. Chargement du modèle E5 et application de LoRA ---
print("Chargement du modèle E5 de base...")
model = SentenceTransformer(config["model_name"])

print("Configuration de LoRA...")
lora_config = LoraConfig(
    r=config["lora_r"],
    lora_alpha=config["lora_alpha"],
    lora_dropout=config["lora_dropout"],
    bias="none",
    target_modules=["query", "key", "value"],
)
model = get_peft_model(model, lora_config)
print("Modèle avec adaptateurs LoRA prêt.")
print_trainable_parameters(model)

# --- 3. Chargement et Division des données ---
try:
    df = pd.read_csv(config["dataset"]).dropna(subset=['Titre', 'Contenu'])
    df_sample = df.sample(n=config["total_sample_size"], random_state=config["random_state"])
    df_train_val, df_test = train_test_split(df_sample, test_size=config["test_size"], random_state=config["random_state"])
    val_split_ratio = config["validation_size"] / (1 - config["test_size"])
    df_train, df_val = train_test_split(df_train_val, test_size=val_split_ratio, random_state=config["random_state"])
    print(f"Entraînement: {len(df_train)}, Validation: {len(df_val)}, Test: {len(df_test)}")
except FileNotFoundError:
    print(f"Erreur : Le fichier '{config['dataset']}' n'a pas été trouvé."); exit()

# --- 4. Définition des Fonctions d'Évaluation ---

def evaluate_triplets_on_split(model_to_eval, dataframe, evaluation_name):
    print(f"\n--- Démarrage Évaluation Triplets: '{evaluation_name}' ---")
    titles, corpus = dataframe['Titre'].tolist(), dataframe['Contenu'].tolist()
    if len(corpus) < 2: return

    title_embeddings = model_to_eval.encode(titles, convert_to_tensor=True, show_progress_bar=True)
    corpus_embeddings = model_to_eval.encode(corpus, convert_to_tensor=True, show_progress_bar=True)

    all_positive_scores, all_negative_scores = [], []
    for idx in range(len(corpus)):
        query_embedding = title_embeddings[idx]
        cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
        other_scores = [(i, s.item()) for i, s in enumerate(cos_scores) if i != idx]
        if not other_scores: continue
        other_scores = sorted(other_scores, key=lambda x: x[1], reverse=True)
        all_positive_scores.append(other_scores[0][1])
        all_negative_scores.append(other_scores[-1][1])

    avg_pos = np.mean(all_positive_scores)
    avg_neg = np.mean(all_negative_scores)
    print(f"--- RÉSULTATS TRIPLETS ({evaluation_name}) ---")
    print(f"Similarité Positive Moyenne: {avg_pos:.4f}")
    print(f"Similarité Négative Moyenne: {avg_neg:.4f}")
    print(f"Delta de Similarité: {avg_pos - avg_neg:.4f}")

    del title_embeddings, corpus_embeddings; gc.collect(); torch.cuda.empty_cache()

def evaluate_retrieval_on_split(model_to_eval, dataframe, evaluation_name):
    print(f"\n--- Démarrage Évaluation Recherche: '{evaluation_name}' ---")
    titles, corpus = dataframe['Titre'].tolist(), dataframe['Contenu'].tolist()
    if len(corpus) < 2: return

    title_embeddings = model_to_eval.encode(titles, convert_to_tensor=True, show_progress_bar=True)
    corpus_embeddings = model_to_eval.encode(corpus, convert_to_tensor=True, show_progress_bar=True)

    cos_scores = util.cos_sim(title_embeddings, corpus_embeddings)
    ranks, hits_at_1, hits_at_5, hits_at_10 = [], 0, 0, 0
    for i in range(len(titles)):
        sorted_indices = torch.argsort(cos_scores[i], descending=True)
        try:
            rank = (sorted_indices == i).nonzero(as_tuple=True)[0].item() + 1
            ranks.append(rank)
            if rank <= 1: hits_at_1 += 1
            if rank <= 5: hits_at_5 += 1
            if rank <= 10: hits_at_10 += 1
        except (IndexError, RuntimeError):
            ranks.append(float('inf'))

    mrr = np.mean([1.0/r for r in ranks if r != float('inf')])
    r1, r5, r10 = hits_at_1/len(titles), hits_at_5/len(titles), hits_at_10/len(titles)
    print(f"--- RÉSULTATS RECHERCHE ({evaluation_name}) ---")
    print(f"R@1: {r1:.4f} | R@5: {r5:.4f} | R@10: {r10:.4f} | MRR: {mrr:.4f}")

    del title_embeddings, corpus_embeddings; gc.collect(); torch.cuda.empty_cache()

# --- 5. ÉVALUATION DE BASE (AVANT FINE-TUNING) ---
print("\n" + "="*50)
print(" " * 10 + "ÉVALUATION DU MODÈLE DE BASE")
print("="*50)
evaluate_triplets_on_split(model, df_test, "Baseline")
evaluate_retrieval_on_split(model, df_test, "Baseline")

# --- 6. PRÉPARATION DES DONNÉES D'ENTRAÎNEMENT ---
print("\n--- PRÉPARATION DES DONNÉES POUR LE FINE-TUNING ---")
train_examples = []
train_data = df_train.to_dict('records')
for i, item in enumerate(train_data):
    anchor, positive = item['Titre'], item['Contenu']
    neg_idx = i
    while neg_idx == i: neg_idx = random.randint(0, len(train_data) - 1)
    negative = train_data[neg_idx]['Contenu']
    train_examples.append(InputExample(texts=[anchor, positive, negative]))
print(f"{len(train_examples)} triplets d'entraînement créés.")

# --- 7. FINE-TUNING DU MODÈLE AVEC LORA ---
print("\n--- DÉBUT DU FINE-TUNING AVEC LORA ---")
train_loss = TripletLoss(model=model)
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=config["batch_size"])
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=config["num_epochs"],
    optimizer_params={'lr': config["learning_rate"]},
    warmup_steps=100,
    output_path='./fine_tuned_model',
    show_progress_bar=True
)
print("--- FIN DU FINE-TUNING ---")

del train_examples, train_dataloader, train_loss; gc.collect(); torch.cuda.empty_cache()

# --- 8. ÉVALUATION FINALE (APRÈS FINE-TUNING) ---
print("\n" + "="*50)
print(" " * 8 + "ÉVALUATION DU MODÈLE FINE-TUNÉ")
print("="*50)
evaluate_triplets_on_split(model, df_test, "Fine-tuned")
evaluate_retrieval_on_split(model, df_test, "Fine-tuned")

print("\n--- EXPÉRIENCE TERMINÉE ---")

Chargement du modèle E5 de base...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

--- RÉSULTATS TRIPLETS (Fine-tuned) ---
Similarité Positive Moyenne: 0.9855
Similarité Négative Moyenne: 0.9032
Delta de Similarité: 0.0823

--- Démarrage Évaluation Recherche: 'Fine-tuned' ---


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

--- RÉSULTATS RECHERCHE (Fine-tuned) ---
R@1: 0.0100 | R@5: 0.0350 | R@10: 0.0700 | MRR: 0.0361

--- EXPÉRIENCE TERMINÉE ---


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch
import numpy as np

# --- 1. CHARGEMENT DES MODÈLES ET DES DONNÉES DE TEST ---

# Nom du modèle original
BASE_MODEL_NAME = 'intfloat/multilingual-e5-large'
# Chemin où votre modèle fine-tuné a été sauvegardé
FINETUNED_MODEL_PATH = './fine_tuned_model'

print("Chargement du modèle de BASE...")
base_model = SentenceTransformer(BASE_MODEL_NAME)

print("Chargement du modèle FINE-TUNÉ...")
finetuned_model = SentenceTransformer(FINETUNED_MODEL_PATH)

# On charge le jeu de données COMPLET pour avoir un corpus de recherche plus large
# (Assurez-vous que 'FrenchNews.csv' est accessible)
print("Chargement du corpus de recherche...")
df_corpus = pd.read_csv('FrenchNews.csv').dropna(subset=['Titre', 'Contenu']).reset_index(drop=True)

# Pour la performance, nous allons rechercher parmi un échantillon de 1000 articles
# Vous pouvez augmenter ce nombre si votre machine le permet
df_corpus_sample = df_corpus.sample(n=1000, random_state=42)

corpus_contents = df_corpus_sample['Contenu'].tolist()
corpus_titles = df_corpus_sample['Titre'].tolist()

print(f"Création des embeddings du corpus pour le modèle FINE-TUNÉ (cela peut prendre un moment)...")
corpus_embeddings_finetuned = finetuned_model.encode(corpus_contents, convert_to_tensor=True, show_progress_bar=True)

print(f"Création des embeddings du corpus pour le modèle de BASE...")
corpus_embeddings_base = base_model.encode(corpus_contents, convert_to_tensor=True, show_progress_bar=True)


# --- 2. FONCTION DE TEST ---

def test_model_with_query(query, model, corpus_embeddings, top_k=5):
    """
    Prend une requête, un modèle et un corpus d'embeddings, et affiche les k meilleurs résultats.
    """
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Calcul des similarités
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]

    # Tri des résultats
    top_results = torch.topk(cos_scores, k=top_k)

    print(f"\nRecherche pour la requête : '{query}'")
    for score, idx in zip(top_results[0], top_results[1]):
        # Récupérer le titre correspondant à l'index trouvé
        matched_title = corpus_titles[idx]
        print(f"  - Score: {score.item():.4f} | Titre: {matched_title}")

# --- 3. LISTE DE REQUÊTES À TESTER ---

# Choisissez des requêtes qui testent bien les capacités du modèle
queries_to_test = [
    # Requête spécifique sur un sujet financier
    "Quelles sont les prévisions de croissance pour la France ?",

    # Requête sur un sujet politique/social
    "Débat sur la réforme des retraites",

    # Requête plus ambiguë qui peut avoir plusieurs sens
    "Les résultats du marché à Paris",

    # Requête avec des mots-clés de votre dataset
    "Inquiétudes sur le secteur automobile après le dieselgate",

    # Requête complètement hors-domaine pour voir comment il réagit
    "La meilleure recette de crêpes"
]

# --- 4. EXÉCUTION DE LA COMPARAISON ---

for query in queries_to_test:
    print("\n" + "="*70)
    print(f"REQUÊTE : \"{query}\"")
    print("="*70)

    print("\n--- Résultats du Modèle de BASE ---")
    test_model_with_query(query, base_model, corpus_embeddings_base)

    print("\n--- Résultats du Modèle FINE-TUNÉ ---")
    test_model_with_query(query, finetuned_model, corpus_embeddings_finetuned)

    print("\n" + "="*70)

Chargement du modèle de BASE...
Chargement du modèle FINE-TUNÉ...


Some weights of the model checkpoint at ./fine_tuned_model were not used when initializing XLMRobertaModel: ['encoder.layer.0.attention.self.key.base_layer.bias', 'encoder.layer.0.attention.self.key.base_layer.weight', 'encoder.layer.0.attention.self.key.lora_A.default.weight', 'encoder.layer.0.attention.self.key.lora_B.default.weight', 'encoder.layer.0.attention.self.query.base_layer.bias', 'encoder.layer.0.attention.self.query.base_layer.weight', 'encoder.layer.0.attention.self.query.lora_A.default.weight', 'encoder.layer.0.attention.self.query.lora_B.default.weight', 'encoder.layer.0.attention.self.value.base_layer.bias', 'encoder.layer.0.attention.self.value.base_layer.weight', 'encoder.layer.0.attention.self.value.lora_A.default.weight', 'encoder.layer.0.attention.self.value.lora_B.default.weight', 'encoder.layer.1.attention.self.key.base_layer.bias', 'encoder.layer.1.attention.self.key.base_layer.weight', 'encoder.layer.1.attention.self.key.lora_A.default.weight', 'encoder.layer.

Chargement du corpus de recherche...
Création des embeddings du corpus pour le modèle FINE-TUNÉ (cela peut prendre un moment)...


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Création des embeddings du corpus pour le modèle de BASE...


Batches:   0%|          | 0/32 [00:00<?, ?it/s]


REQUÊTE : "Quelles sont les prévisions de croissance pour la France ?"

--- Résultats du Modèle de BASE ---

Recherche pour la requête : 'Quelles sont les prévisions de croissance pour la France ?'
  - Score: 0.8576 | Titre: La balance des paiements française en juillet
  - Score: 0.8474 | Titre: L'économie française rebondira un peu plus que prévu cette année, selon la Banque de France
  - Score: 0.8274 | Titre: Présidentielle russe : les chiffres clés de l'économie
  - Score: 0.8272 | Titre: La France face au défi du vieillissement de sa population
  - Score: 0.8266 | Titre: Près d'un Français sur trois souffre d'hypertension, dont la moitié qui l'ignore

--- Résultats du Modèle FINE-TUNÉ ---

Recherche pour la requête : 'Quelles sont les prévisions de croissance pour la France ?'
  - Score: 0.9995 | Titre: Vacances : 65 % des Français ont déjà renoncé à partir faute d'argent
  - Score: 0.9994 | Titre: Jean-Baptiste Lemoyne :  Nous espérons pouvoir accélérer la réouverture des resta